<a href="https://colab.research.google.com/github/JstnClmnt/NLP-Text-Summarization/blob/master/NLP_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Article Text Summarization





### Import Libraries


In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import re
np.random.seed(1)

### Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


### Assign to Data Frame

In [70]:
import pandas as pd
df=pd.read_csv("/content/drive/Team Drives/AI Lords/NLP-Text-Summarization/data/news_summary_more.csv",encoding="latin-1")
df.head()


,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [0]:
df['headlines'] = df['headlines'].str.replace("[.,]", " ")
df['text'] = df['text'].str.replace("[.,]", " ")

In [85]:
df["text"][1]

"Kunal Shah's credit card bill payment platform  CRED  gave users a chance to win free food from Swiggy for one year  Pranav Kaushik  a Delhi techie  bagged this reward after spending 2000 CRED coins  Users get one CRED coin per rupee of bill paid  which can be used to avail rewards from brands like Ixigo  BookMyShow  UberEats  Cult Fit and more "

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df["text"], df["headlines"], test_size=0.20, random_state=42)

In [106]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def remove_special_characters(text):
    text = str(text)
    text=text.split(" ")
    words=[]
    for t in text:
      words.append(t)
    text=" ".join(words)
    text=re.sub("what's","what is ",text)
    text=re.sub("it's","it is ",text)
    text=re.sub("\'ve"," have ",text)
    text=re.sub("i'm","i am ",text)
    text=re.sub("\'re"," are ",text)
    text=re.sub("can't"," cannot ",text)
    text=re.sub("n't"," not ",text)
    text=re.sub("\'d"," would ",text)
    text=re.sub("\'s"," ",text)
    text=re.sub("\'"," ",text)
    text=re.sub("\""," ",text)
    text=re.sub("\'ll"," will ",text)
    text=re.sub(" e g "," eg ",text)
    text=re.sub("e-mail","email",text)
    text=re.sub("9\\/11"," 911 ",text)
    text=re.sub(" u.s"," american ",text)
    text=re.sub(" u.n"," united nations ",text)
    text=re.sub("\n"," ",text)
    text=re.sub(":"," ",text)
    text=re.sub("\_"," ",text)
    text=re.sub("\d+"," ",text)
    text=re.sub("[$#@%&*!~?%{}()]"," ",text)
    return word_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [107]:
X_train_tokenized=[remove_special_characters(i) for i in X_train]
X_test_tokenized=[remove_special_characters(i) for i in X_test]
print(X_train_tokenized[0])

['Hollywood', 'actress', 'Kate', 'Winslet', 'has', 'joined', 'the', 'cast', 'of', 'Titanic', 'director', 'James', 'Cameron', 'upcoming', 'films', 'in', 'the', 'Avatar', 'franchise', 'making', 'this', 'their', 'first', 'venture', 'together', 'after', 'years', 'of', 'Titanic', 'release', 'Cameron', 'said', 'Kate', 'and', 'I', 'have', 'been', 'looking', 'for', 'something', 'to', 'do', 'together', 'since', 'our', 'collaboration', 'on', 'Titanic', 'I', 'can', 'not', 'wait', 'to', 'see', 'her', 'bring', 'the', 'character', 'of', 'Ronal', 'to', 'life']


In [108]:
y_train_tokenized=[remove_special_characters(i) for i in y_train]
y_test_tokenized=[remove_special_characters(i) for i in y_test]
print(y_train_tokenized[0])

['Kate', 'Winslet', 'to', 'work', 'with', 'Titanic', 'maker', 'after', 'years']


In [0]:
words = set([])
for s in X_train_tokenized:
    for w in s:
        words.add(w.lower())
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

In [111]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2019-05-16 05:50:16--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-05-16 05:50:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  16.1MB/s    in 67s     

2019-05-16 05:51:23 (12.3 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [112]:
!unzip "glove.6B.zip"

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [115]:
train_sentences_X, test_sentences_X,train_sentences_Y,test_sentences_Y = [], [], [], []

EMB_DIM=300
num_words=len(word2index)+1
print("Number of Words:"+str(num_words))

for s in X_train_tokenized:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
    
for s in X_test_tokenized:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
  
for s in y_train_tokenized:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_Y.append(s_int)
    
for s in y_test_tokenized:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_Y.append(s_int)


MAX_LENGTH = len(max(train_sentences_X, key=len))
print("Max Length: "+str(MAX_LENGTH))  # 271

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
#y_train=to_categorical(y_train)
#y_test=to_categorical(y_test)
print(train_sentences_X[0])
print(train_sentences_Y[0])

Number of Words:84640
Max Length: 88
[69161 22536  9177 41056 79623 35987 54151 67350 12012  4645 15799 38573
  2605  5961 56547 83068 54151 74550 48976 51739 39578 26251   831 27708
 71969 69489 77000 12012  4645 41791  2605 34393  9177 70676 10976 81506
 58964 26797 15986  3390 70630 60261 71969 28796 73170 63307 83989  4645
 10976  7455 48038  6186 70630 36878 46073 71439 54151 62929 12012 46026
 70630 45654     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[9177, 41056, 70630, 12997, 80457, 4645, 47262, 69489, 77000]


In [0]:
embedding_matrix=np.zeros((num_words,EMB_DIM))
#print(word2index)
for word,i in word2index.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector